# 视频过滤器可视化测试

这个Notebook用于可视化测试单个视频文件的过滤过程，包括：
1. 分辨率检测
2. 人脸检测和可视化
3. FaceXFormer年龄估计
4. 最终过滤结果

In [ ]:
# 导入必要的库
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import os
from simple_video_filter import SimpleVideoFilter
import warnings
warnings.filterwarnings('ignore')

# 设置matplotlib中文字体
plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 库导入完成")

In [ ]:
# 配置测试参数
VIDEO_PATH = "rawdata/test/111.mp4"  # 修改这里来测试不同的视频
SAMPLE_FRAMES = 5  # 采样帧数
MAX_DISPLAY_FRAMES = 3  # 最多显示的帧数

print(f"测试视频: {VIDEO_PATH}")
print(f"采样帧数: {SAMPLE_FRAMES}")
print(f"显示帧数: {MAX_DISPLAY_FRAMES}")

In [ ]:
# 初始化视频过滤器
print("正在初始化FaceXFormer视频过滤器...")
filter_tool = SimpleVideoFilter(use_facexformer=True)
print("✓ 过滤器初始化完成")

## 1. 视频基本信息检测

In [ ]:
# 检测视频基本信息
def get_video_info(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return None
    
    info = {
        'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)),
        'fps': cap.get(cv2.CAP_PROP_FPS),
        'frame_count': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        'duration': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) / cap.get(cv2.CAP_PROP_FPS)
    }
    cap.release()
    return info

# 获取视频信息
video_info = get_video_info(VIDEO_PATH)

if video_info:
    print("📹 视频基本信息:")
    print(f"  分辨率: {video_info['width']}x{video_info['height']}")
    print(f"  帧率: {video_info['fps']:.2f} FPS")
    print(f"  总帧数: {video_info['frame_count']}")
    print(f"  时长: {video_info['duration']:.2f} 秒")
    
    # 分辨率检测
    resolution_passed = video_info['width'] >= 720 and video_info['height'] >= 1080
    print(f"\n🔍 分辨率检测: {'✓ 通过' if resolution_passed else '✗ 不通过'} (要求: ≥720x1080)")
else:
    print("❌ 无法读取视频文件")

## 2. 人脸检测可视化

In [ ]:
# 提取和显示采样帧
def extract_sample_frames(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, min(num_frames, total_frames), dtype=int)
    
    frames = []
    for frame_idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            frames.append((frame_idx, frame))
    
    cap.release()
    return frames

# 人脸检测和可视化
def detect_and_visualize_faces(frame, face_cascade):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    # 在图像上绘制人脸框
    frame_with_faces = frame.copy()
    valid_faces = 0
    
    for (x, y, w, h) in faces:
        if w >= 256 and h >= 256:
            # 有效人脸 - 绿色框
            cv2.rectangle(frame_with_faces, (x, y), (x+w, y+h), (0, 255, 0), 3)
            cv2.putText(frame_with_faces, f'Valid: {w}x{h}', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            valid_faces += 1
        else:
            # 无效人脸 - 红色框
            cv2.rectangle(frame_with_faces, (x, y), (x+w, y+h), (0, 0, 255), 2)
            cv2.putText(frame_with_faces, f'Small: {w}x{h}', (x, y-10), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    return frame_with_faces, len(faces), valid_faces

# 提取采样帧
sample_frames = extract_sample_frames(VIDEO_PATH, SAMPLE_FRAMES)
print(f"📸 提取了 {len(sample_frames)} 个采样帧")

In [ ]:
# 显示人脸检测结果
if sample_frames and filter_tool.face_cascade:
    fig, axes = plt.subplots(min(len(sample_frames), MAX_DISPLAY_FRAMES), 1, 
                            figsize=(12, 4*min(len(sample_frames), MAX_DISPLAY_FRAMES)))
    
    if min(len(sample_frames), MAX_DISPLAY_FRAMES) == 1:
        axes = [axes]
    
    total_faces = 0
    total_valid_faces = 0
    
    for i, (frame_idx, frame) in enumerate(sample_frames[:MAX_DISPLAY_FRAMES]):
        frame_with_faces, face_count, valid_face_count = detect_and_visualize_faces(
            frame, filter_tool.face_cascade)
        
        total_faces += face_count
        total_valid_faces += valid_face_count
        
        # 转换BGR到RGB用于matplotlib显示
        frame_rgb = cv2.cvtColor(frame_with_faces, cv2.COLOR_BGR2RGB)
        
        axes[i].imshow(frame_rgb)
        axes[i].set_title(f'Frame {frame_idx}: {face_count} faces detected, {valid_face_count} valid (≥256x256)')
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 人脸检测统计
    face_passed = total_valid_faces > 0
    print(f"\n👥 人脸检测统计:")
    print(f"  总检测人脸数: {total_faces}")
    print(f"  有效人脸数: {total_valid_faces} (≥256x256)")
    print(f"  人脸检测结果: {'✓ 通过' if face_passed else '✗ 不通过'}")
else:
    print("❌ 无法进行人脸检测")

## 3. FaceXFormer年龄估计

In [ ]:
# 年龄估计可视化
def visualize_age_estimation(video_path, filter_tool):
    print("🧠 正在进行FaceXFormer年龄估计...")
    
    # 使用过滤器的年龄检测功能
    age_passed, age_info = filter_tool.check_age(video_path, sample_frames=5)
    
    print(f"\n📊 年龄估计结果:")
    print(f"  估计年龄: {age_info.get('estimated_age', 'N/A')}")
    print(f"  年龄类别: {age_info.get('age_class', 'N/A')}")
    print(f"  检测方法: {age_info.get('method', 'N/A')}")
    print(f"  置信度: {age_info.get('confidence', 'N/A')}")
    print(f"  处理人脸数: {age_info.get('face_count', 'N/A')}")
    
    if 'age_distribution' in age_info:
        print(f"  年龄分布: {age_info['age_distribution']}")
    
    print(f"  年龄检测结果: {'✓ 通过' if age_passed else '✗ 不通过'} (要求: 0-30岁)")
    
    return age_passed, age_info

# 执行年龄估计
if os.path.exists(VIDEO_PATH):
    age_passed, age_info = visualize_age_estimation(VIDEO_PATH, filter_tool)
else:
    print("❌ 视频文件不存在")
    age_passed = False
    age_info = {}

## 4. 年龄分类可视化

In [ ]:
# 年龄分类图表
def plot_age_classification():
    age_classes = ['0-10', '11-20', '21-30', '31-40', '41-50', '51-60', '61-70', '70+']
    pass_status = ['通过', '通过', '通过', '不通过', '不通过', '不通过', '不通过', '不通过']
    colors = ['green' if status == '通过' else 'red' for status in pass_status]
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    bars = ax.bar(age_classes, [1]*8, color=colors, alpha=0.7)
    
    # 标记当前检测结果
    if age_info and 'estimated_age' in age_info:
        current_age = age_info['estimated_age']
        if current_age in age_classes:
            idx = age_classes.index(current_age)
            bars[idx].set_edgecolor('blue')
            bars[idx].set_linewidth(4)
            ax.text(idx, 0.5, f'当前检测\n{current_age}', ha='center', va='center', 
                   fontsize=12, fontweight='bold', color='blue')
    
    ax.set_title('FaceXFormer年龄分类系统 (绿色=通过过滤, 红色=不通过, 蓝色边框=当前检测结果)', fontsize=14)
    ax.set_xlabel('年龄范围', fontsize=12)
    ax.set_ylabel('过滤状态', fontsize=12)
    ax.set_ylim(0, 1.2)
    
    # 添加图例
    from matplotlib.patches import Patch
    legend_elements = [Patch(facecolor='green', alpha=0.7, label='通过 (0-30岁)'),
                      Patch(facecolor='red', alpha=0.7, label='不通过 (31岁以上)')]
    ax.legend(handles=legend_elements, loc='upper right')
    
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

plot_age_classification()

## 5. 最终过滤结果

In [ ]:
# 完整的过滤测试
print("🎯 执行完整的视频过滤测试...")
print("=" * 60)

if os.path.exists(VIDEO_PATH):
    result = filter_tool.process_video(VIDEO_PATH)
    
    print("\n📋 最终过滤结果:")
    print(f"  视频文件: {result['video_name']}")
    print(f"  分辨率检测: {'✓ 通过' if result['resolution_passed'] else '✗ 不通过'} ({result['resolution']})")
    print(f"  人脸检测: {'✓ 通过' if result['face_passed'] else '✗ 不通过'} ({result['face_info']})")
    print(f"  年龄检测: {'✓ 通过' if result['age_passed'] else '✗ 不通过'} ({result['age_info']})")
    print(f"  最终结果: {'🎉 通过所有过滤条件' if result['final_passed'] else '❌ 未通过过滤'}")
    
    # 结果可视化
    fig, ax = plt.subplots(1, 1, figsize=(10, 6))
    
    criteria = ['分辨率\n(≥720x1080)', '人脸检测\n(≥256x256)', '年龄估计\n(0-30岁)', '最终结果']
    results = [result['resolution_passed'], result['face_passed'], 
              result['age_passed'], result['final_passed']]
    colors = ['green' if r else 'red' for r in results]
    
    bars = ax.bar(criteria, [1]*4, color=colors, alpha=0.7)
    
    # 添加结果标签
    for i, (bar, passed) in enumerate(zip(bars, results)):
        ax.text(i, 0.5, '✓ 通过' if passed else '✗ 不通过', 
               ha='center', va='center', fontsize=12, fontweight='bold', color='white')
    
    ax.set_title(f'视频过滤结果总览: {VIDEO_PATH}', fontsize=14, fontweight='bold')
    ax.set_ylabel('过滤状态', fontsize=12)
    ax.set_ylim(0, 1.2)
    
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
else:
    print("❌ 视频文件不存在，无法进行测试")

print("\n" + "=" * 60)
print("测试完成！")